# Textual Analysis
You should create an seperate `yaml` for each of the project you are working on, this is a good practice in general. Thesre migt be weird issues that causes a dependency problem. For the same reason a new `yaml` file would record all the dependencies that would ensure this pipeline works for future references.

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys

current_folder = Path.cwd()
parent_folder = current_folder.parent
sys.path.insert(0, str(parent_folder))
print(parent_folder)

import plotly.io as pio
pio.renderers.keys()
pio.renderers.default = 'notebook' 

import pandas as pd
import numpy as np
import os
import glob
import nltk
nltk.download('punkt')
from utils.clean_text import transform_text

/Users/kevinb/Desktop/dsc80/Chatable-Study-Database


[nltk_data] Downloading package punkt to /Users/kevinb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/kevinb/miniforge3/envs/ucsd_study/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [2]:
current_folder = Path.cwd()
parent_folder = current_folder.parent
os.chdir(parent_folder)
print(parent_folder)

# text data
# fall_2022_text = pd.read_csv('data/2022_fall_text.csv')
# winter_2023_text = pd.read_csv('data/2023_winter_text.csv')
# spring_2023_text = pd.read_csv('data/2023_spring_text.csv')
# summer_2023_text = pd.read_csv('data/2023_summer_text.csv')
# fall_2023_text = pd.read_csv('data/2023_fall_text.csv')
# winter_2024_text = pd.read_csv('data/2024_winter_text.csv')
# spring_2024_text = pd.read_csv('data/2024_spring_text.csv')
# summer_2024_text = pd.read_csv('data/2024_summer_text.csv')
# fall_2024_text = pd.read_csv('data/2024_fall_text.csv')
# winter_2025_text = pd.read_csv('data/2025_winter_text.csv')
# text = pd.concat([fall_2022_text, winter_2023_text, spring_2023_text, summer_2023_text, fall_2023_text, winter_2024_text, spring_2024_text, summer_2024_text, fall_2024_text, winter_2025_text], axis=0)

data_dir = 'data'
file_pattern = os.path.join(data_dir, '*_text.csv')
csv_files = glob.glob(file_pattern)
csv_files.sort()
df_list = [pd.read_csv(file) for file in csv_files]
text = pd.concat(df_list, axis=0, ignore_index=True)
print(f"Total Texts: {text.shape[0]}")

/Users/kevinb/Desktop/dsc80/Chatable-Study-Database
Total Texts: 763


In [3]:
clean_text = text.pipe(transform_text)
clean_text

,Time,Quarter,Month,Study Materials
0,2005-02-23,2005Q1,February,2005-02-23 | 2005Q1 | February | SOO formulati...
1,2022-02-14,2022Q1,February,2022-02-14 | 2022Q1 | February | Book read (30...
2,2022-02-15,2022Q1,February,2022-02-15 | 2022Q1 | February | Nutrition rea...
3,2022-10-21,2022Q4,October,2022-10-21 | 2022Q4 | October |
4,2022-10-22,2022Q4,October,2022-10-22 | 2022Q4 | October | After consecut...
...,...,...,...,...
702,2025-03-16,2025Q1,March,2025-03-16 | 2025Q1 | March | Math 189 notes +...
703,2025-03-17,2025Q1,March,2025-03-17 | 2025Q1 | March | Cogs 101a review...
704,2025-03-18,2025Q1,March,2025-03-18 | 2025Q1 | March | Dress + driving ...
705,2025-03-19,2025Q1,March,2025-03-19 | 2025Q1 | March | Get stuff with T...


In [4]:
pre_process = (clean_text['Study Materials']
              .str.lower()
              .str.replace(r'\([\d]*m\)','',regex=True)
              .str.replace(',','')
              .str.strip())

corpus = ' '.join(pre_process.astype(str).to_list())
tokens = nltk.tokenize.word_tokenize(corpus, language='english')
len(tokens)

21178

# Making Chat Familier With My Data

Using either of `sentence-transformer`, `nltk`, `openai`, `langchain`, or related stuff has many dependency issue if just pyt in a big environment, need to have a seperated contained environment.

In [5]:
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import faiss
import ast
import openai
import os

nltk.download('punkt')
model = SentenceTransformer('all-MiniLM-L6-v2')

[nltk_data] Downloading package punkt to /Users/kevinb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from utils.clean_text import split_text_nltk, get_similar_chunks, generate_response

documents = clean_text['Study Materials'].tolist()
all_chunks = []
for doc in documents:
    chunks = split_text_nltk(doc)
    all_chunks.extend(chunks)

print(f"Total chunks created: {len(all_chunks)}")
embeddings = model.encode(all_chunks, show_progress_bar=True, convert_to_tensor=False)

embedding_df = pd.DataFrame({
    'chunk': all_chunks,
    'embedding': embeddings.tolist(),
    'quarter': clean_text['Quarter'],
    'time': clean_text['Time'],
    'month':clean_text['Month']
})

embedding_df.to_csv('embeddings/embeddings.csv', index=False)
print("Embeddings saved to embeddings.csv")

Total chunks created: 707


Batches: 100%|██████████| 23/23 [00:03<00:00,  6.39it/s]

Embeddings saved to embeddings.csv


In [7]:
embedding_df = pd.read_csv('embeddings/embeddings.csv')
embedding_df['embedding'] = embedding_df['embedding'].apply(ast.literal_eval)

# Convert embeddings to a NumPy array of type float32
embeddings = np.array(embedding_df['embedding'].tolist()).astype('float32')

# Initialize FAISS index and using L2 distance, can also use cosine similarity
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings)
print(f"FAISS index has {index.ntotal} vectors.")

FAISS index has 707 vectors.


In [8]:
embedding_df

,chunk,embedding,quarter,time,month
0,2005-02-23 | 2005Q1 | February | SOO formulati...,"[0.024124285206198692, 0.031140342354774475, 0...",2005Q1,2005-02-23,February
1,2022-02-14 | 2022Q1 | February | Book read (30...,"[-0.03661457449197769, -0.006186727900058031, ...",2022Q1,2022-02-14,February
2,2022-02-15 | 2022Q1 | February | Nutrition rea...,"[-0.05766584724187851, -0.050128839910030365, ...",2022Q1,2022-02-15,February
3,2022-10-21 | 2022Q4 | October |,"[-0.03648470714688301, -0.021536782383918762, ...",2022Q4,2022-10-21,October
4,2022-10-22 | 2022Q4 | October | After consecut...,"[-0.08645892143249512, -0.028704402968287468, ...",2022Q4,2022-10-22,October
...,...,...,...,...,...
702,2025-03-16 | 2025Q1 | March | Math 189 notes +...,"[-0.08612487465143204, 0.018114281818270683, 0...",2025Q1,2025-03-16,March
703,2025-03-17 | 2025Q1 | March | Cogs 101a review...,"[-0.08891618251800537, 0.012606654316186905, -...",2025Q1,2025-03-17,March
704,2025-03-18 | 2025Q1 | March | Dress + driving ...,"[-0.056250184774398804, 0.022247901186347008, ...",2025Q1,2025-03-18,March
705,2025-03-19 | 2025Q1 | March | Get stuff with T...,"[-0.06770867854356766, 0.03140786662697792, -0...",2025Q1,2025-03-19,March


In [9]:
api_key = "..." #os.getenv('OPENAI_API_KEY')

In [10]:
# user_prompt = "What did I mainly do in 2022 fall quarter?"
# similar_chunks = get_similar_chunks(user_prompt, index, embedding_df, top_k=5)
# generate_response(user_prompt, similar_chunks, api_key=api_key)

In [11]:
# user_prompt = "What did I mainly do in 2023 fall quarter?"
# similar_chunks = get_similar_chunks(user_prompt, index, embedding_df, top_k=5)
# generate_response(user_prompt, similar_chunks, api_key=api_key)

In [12]:
# user_prompt = "What did I mainly do in 2024 fall quarter?"
# similar_chunks = get_similar_chunks(user_prompt, index, embedding_df, top_k=5)
# generate_response(user_prompt, similar_chunks, api_key=api_key)